<a href="https://colab.research.google.com/github/suzukimain/diffusers_in_Colab/blob/main/Image_generation_ver_Bete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#重要な注意点(Important Notes)：
 **・商用利用はご遠慮ください。**

 (Please do not use for commercial purposes.)

**・画像生成によって起こった問題について、私は一切責任を負いません。**

(I am not responsible for any problems caused by the image generation.)

主に以下の内容が含まれます。

* モデルを使用して、違法または有害な出力またはコンテンツを意図的に作成または共有することはできません。

* モデル作成者は、利用者が生成した出力に対していかなる権利も主張せず、それらを利用者は自由に使用することができます。また、Licenseの規定に違反していない使用について、利用者は説明責任を負います。

* 重みを再配布し、モデルを商用またはサービスとして使用することができます。その場合、ライセンスに記載されているものと同じ制限を含めて、The CreativeML OpenRAIL Licenseのコピーをすべての利用者と共有する必要があることに注意してください。


上記の内容と、最下部に記載されている利用規約に同意できない方は、本画像生成ブックの使用はご遠慮ください。

(If you do not agree to the above contents and the terms of use listed at the bottom, please do not use this image generation book.)

追記:
本ノートブックを使用した時点で重要な注意点と、利用規約に同意したとみなします。

(P.S. By using this notebook, you agree to the important notes and terms of use.)

>ランタイムの変更方法

ランタイム>ランタイムのタイプを変更>Noneを押す>GPUを選択>決定.
(Runtime > Change runtime type > Select 'GPU' after clicking 'None' > Save)

In [ ]:
#@title  #🔽実行する時このような三角のボタンを押してください{ run: "auto", display-mode: "form"}
#@markdown #Step1.セットアップ (Setup)

import locale
locale.getpreferredencoding = lambda: "UTF-8"

Google_driveに接続 = False  # @param {type:"boolean"}
#@markdown Googleドライブに保存したいときにチェックをしてください
if Google_driveに接続:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
  except:
    print("ドライブへの接続に失敗しました")
    pass
import os

try:
  import torch
  import diffusers
  import transformers
  import accelerate
  import scipy
  import safetensors
  import ftfy
  import regex
  import tqdm
  import xformers
  import sentencepiece
  import pysbd
  import huggingface_hub


except:
  try:
    !pip install torch==2.0.1+cu118 diffusers==0.16.1 transformers==4.29.2 accelerate==0.19.0 scipy==1.10.1 safetensors==0.3.1 ftfy==6.1.1 regex==2022.10.31 tqdm==4.65.0 scipy==1.10.1 sentencepiece==0.1.99 pysbd==0.3.4 xformers huggingface_hub -q
    !git clone https://github.com/huggingface/diffusers.git
    import torch
    import diffusers
    import transformers
    import accelerate
    import scipy
    import safetensors
    import ftfy
    import regex
    import tqdm
    import sentencepiece
    import pysbd
    import huggingface_hub
  except:
    raise RuntimeError("ランタイムをGPUに変更お願いします。またはページの再読み込みをお願いします")
#try:
# import clip
#except ModuleNotFoundError:
#  !pip install git+https://github.com/openai/CLIP.git -q
#  import clip
diffusers_path = "/content/diffusers"
if not os.path.exists(diffusers_path):
  !git clone https://github.com/huggingface/diffusers.git -q

import requests
import spacy
import codecs
import pickle
import torch
import random
import numpy as np
import transformers
import sentencepiece
from huggingface_hub import hf_hub_download
import ipywidgets as widgets
from PIL import Image
from datetime import datetime
from transformers import pipeline
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline
from torch import Generator, autocast
from IPython.display import display, Markdown
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from diffusers import EulerDiscreteScheduler, StableDiffusionImg2ImgPipeline
from diffusers import DiffusionPipeline, AutoencoderKL, EulerAncestralDiscreteScheduler
from diffusers.models import AutoencoderKL

m_dir = '/content/モデルフォルダ'
if not os.path.exists(m_dir):
  os.makedirs(m_dir)
def model_paths(m_dir):
  file_name = os.listdir(m_dir)[0] # フォルダ内の最初のファイル名を取得
  model_path=os.path.join(m_dir)
  return model_path

Prompt_tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
Prompt_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
Prompt_model = GPT2LMHeadModel.from_pretrained('FredZhang7/anime-anything-promptgen-v2')

#hf_hub_download(repo_id="gsdf/EasyNegative", filename="EasyNegative.pt", repo_type="dataset",cache_dir="/content")


#model_path=model_paths(m_dir,file_name)
#pipe = StableDiffusionPipeline.from_ckpt(model_path)
print("\033[32mセットアップが完了しました。必要に応じて '/content/入力するモデルファイル'に使いたいモデルファイルを入れてください\033[0m")

In [ ]:
#@title  #Step2.モードの選択 (Mode Selection){ run: "auto", display-mode: "form"}


# @markdown >使用するモデルの切り替え (model change)

model_select = "stable diffusion-v2.1(basic)" #@param ["stable diffusion-v2.1(basic)", "waifu diffusion-v1.4(Anime)(better)","Anything-v3.0(Anime)","anything-midjourney-v-4-1(Anime)","Anything-v4.5(Anime)","SukiyakiMix-v1.0(Anime)","Counterfeit-V2.0(Anime)","Counterfeit-V2.5(Anime)","ACertainThing(Anime)","anime-anything-promptgen-v2(Anime)","Counterfeit(Anime)","7th_Layer(Anime)","EimisAnimeDiffusion-v2.0(Anime)","Riga_Collection(Anime)","anime-kawai-diffusion(Anime)","JWST-Deep-Space-diffusion(scenery)","Custom"]

#@markdown Customモードは "入力するモデルフォルダ" に１つのモデルファイルを入れてください

# @markdown >モードの切り替え (mode change)
mode_select = "Quick" #@param ["Nomal(better)","Quick"]

#@markdown * "Nomal" 品質と生成時間のバランス重視です

#@markdown * "Quick" 少し品質が低下する代わりに生成時間が大幅に短縮します


入力形式 = "text_only(better)" #@param ["text_only(better)", "image_and_text"]

# @markdown

# @markdown  **チェックをつけると接続に必要な権限を求めるようになります**

# @markdown (Checking this box will prompt you for the necessary permissions to connect.)

# @markdown

# @markdown >フィルターを調整

# @markdown **注意事項 : 変更する時は注意して下さい**

Filter_off = False  # @param {type:"boolean"}


# @markdown 追記:人が映り込んだだけでフィルタリングされることがあったので追加しました

def result():
  if Filter_off == False:
     filter_level = "フィルターの強度:通常"
  else:
     filter_level = "\033[33mフィルターの強度:無効\033[0m"
  return filter_level

def safe():
  if Filter_off ==False:
    safety_checker = ""

  else:
    safety_checker = None
  return safety_checker


import requests
import spacy
import codecs
import pickle
import torch
import random
import numpy as np
import transformers
import sentencepiece
import ipywidgets as widgets
from PIL import Image
from datetime import datetime
from transformers import pipeline
from torch import Generator, autocast
from IPython.display import display, Markdown
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from diffusers import EulerDiscreteScheduler, StableDiffusionImg2ImgPipeline
from diffusers import DiffusionPipeline, AutoencoderKL, EulerAncestralDiscreteScheduler
from diffusers.models import AutoencoderKL
#from diffusers.loaders import pipe_setup, pipe_load_textual_inversion

device = "cuda" if torch.cuda.is_available() else "cpu"
#model, preprocess = clip.load("ViT-B/32", device=device)

# 使用するモデルを設定
if model_select =="Custom":
   model_id="Custom"
   model_name="Custom"

elif model_select == "stable diffusion-v2.1(basic)":
   model_id = "stabilityai/stable-diffusion-2-1"
   model_name = "stable_diffusion-v2.1"
   g_scale ="7"

elif model_select == "waifu diffusion-v1.4(Anime)(better)":
   model_id = "hakurei/waifu-diffusion"
   model_name = "waifu_diffusion-v1.4"
  # g_scale =

elif model_select == "Anything-v3.0(Anime)":
   model_id = "Linaqruf/anything-v3.0"
   model_name = "Anything-v3.0"
#   g_scale =

elif model_select == "anything-midjourney-v-4-1(Anime)":
  model_id = "Joeythemonster/anything-midjourney-v-4-1"
  model_name ="anything-midjourney-v-4-1"
   # g_scale =

elif model_select == "Anything-v4.5(Anime)":
   model_id = "Airic/Anything-V4.5"
   model_name = "Anything-v4.5"
 #  g_scale =

elif model_select == "ACertainThing(Anime)":
   model_id ="JosephusCheung/ACertainThing"
   model_name="JosephusCheung/ACertainThing"
 #  g_scale=

elif model_select == "Counterfeit-V2.0(Anime)":
   model_id = "gsdf/Counterfeit-V2.0"
   model_name= "Counterfeit-V2.0"
#   g_scale =

elif model_select =="Counterfeit-V2.5(Anime)":
   mopdel_id = "gsdf/Counterfeit-V2.5"
   model_name ="Counterfeit-V2.5"
elif model_select == "SukiyakiMix-v1.0(Anime)":
  model_id = "Vsukiyaki/SukiyakiMix-v1.0"
  model_name = "SukiyakiMix-v1.0"
#  g_scale =

elif model_select == "Counterfeit(Anime)":
   model_id = "gsdf/Counterfeit-V2.5"
   model_name ="Counterfeit-V2.5"
 #  g_scale =

elif model_select == "7th_Layer(Anime)":
   model_id = "syaimu/7th_test"
   model_name = "7th_Layer"
 #  g_scale =

elif model_select == "EimisAnimeDiffusion-v2.0(Anime)":
   model_id = "eimiss/EimisAnimeDiffusion_2.0v"
   model_name = "EimisAnimeDiffusion-v2.0"
 #  g_scale =

elif model_select=="Riga_Collection(Anime)":
   mdoel_id ="natsusakiyomi/Riga_Collection"
   model_name="Riga_Collection"
  # g_scale =

# モードの切り替え (mode change)
elif model_select =="anime-kawai-diffusion(Anime)":
  model_id="Ojimi/anime-kawai-diffusion"
  model_name="anime-kawai-diffusion"
 # g_scale =

#elif model_select =="JWST-Deep-Space-diffusion(scenery)":
#  model_id="dallinmackay/JWST-Deep-Space-diffusion"
#  model_name="JWST-Deep-Space-diffusion"
 # g_scale =

#dtype = torch.float16 if mode_select == "Quick" else torch.float32

elif model_select == "anything-midjourney-v-4-1(Anime)":
  model_id = "Joeythemonster/anything-midjourney-v-4-1"
  model_name ="anything-midjourney-v-4-1"
   # g_scale =

diffuser_kwargs = {
   "negative_prompt": "worst quality, low quality, medium quality, deleted, lowres, comic, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry",
   "seed": 72566,
   "temperature": 0.8,
   "truncation": 0.8,
   "diffusion_steps": 500
}



def pipe_setup():
  if not model_id=="Custom":
    #vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse")
    if 入力形式 == "text_only(better)":
      if Filter_off==True:
        # StableDiffusion(メインのモジュール)パイプライン設定
        # もしも何かあったら"torch_dtype=torch.float16"をつける
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id, torch_dtype=torch.float16 if mode_select == "Quick" else torch.float32,
            safety_checker= None,
            clip_model=lambda x: model.encode_image(x).float(),
            #vae=vae
        )
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

      else:
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id, torch_dtype=torch.float16 if mode_select == "Quick" else torch.float32,
            clip_model=lambda x: model.encode_image(x).float(),
           # vae=vae
        )
        pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    else:
      if Filter_off==True:
        #StableDiffusionパイプライン設定
        scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
      else:
        scheduler = EulerDiscreteScheduler.from_pretrained(model_id,safety_checker= None, subfolder="scheduler")
  else:
    try:
      model_path=model_paths(m_dir)
      pipe = StableDiffusionPipeline.from_ckpt(model_path)
    except:
      raise ModuleNotFoundError("'入力するモデルファイル'からモデルを読み込めませんでした。モデルファイルが存在しないか対応していない形式の可能性があります")
  #pipe.load_textual_inversion("EasyNegative-test", weight_name="/content/EasyNegative.safetensors", token="EasyNegative")
  pipe = pipe.to(device)
  return pipe

def ima3ima(modl_id):
  model_T="stable_diffusion_txt2img"+model_id
  return model_T
ima3ima(model_id)
# txt2img モードで Pipeline オブジェクトを作成
model_T = StableDiffusionPipeline.from_pretrained(model_id)

# generate text using fine-tuned model
nlp = pipeline('text-generation', model=Prompt_model, tokenizer=Prompt_tokenizer)

pipe=pipe_setup()
filter_level=result()
print(filter_level)

en_save_dir ="/content/datasets--gsdf--EasyNegative/snapshots/60067b257337df8d7879142d870944fe4c6ab20d/EasyNegative.pt"


#emb_params = torch.load(en_save_dir) # ptファイルを読み込んでemb_paramsに代入
#emb_params = emb_params.reshape(1024) # emb_paramsのテンソルの形状を[1024]に変更


# ここにreshapeを追加
#emb_params = emb_params.reshape(768)

#pipe.load_textual_inversion(en_save_dir, token="EasyNegative")#\033[31mが赤、\033[33mが黄色、\033[34mが青、\033[32mが緑、\033[0mが白
# "\033[32m" は緑色に変更するための"ANSI Escape Code"であり、"\033[0m"はデフォルトの文字色に戻すためのコードです。
print("\033[32m画像生成の準備が出来たので、手順3に移ってください。\033[0m")
print("\033[32m(Now that the image generation is ready, please proceed to step 3.)\033[0m")

In [ ]:
#@title  #Step3.画像の生成  (iamge generation){display-mode: "form"}

#@markdown #自動生成

自動で条件を決めて生成 = False #@param {type:"boolean"}

#@markdown **操作がよくわからない方はチェックをつけて下さい。自動で生成します**

#@markdown >この機能の詳細

#@markdown  * 詳細設定を推奨の値に設定

#@markdown  * プロンプトが入力されていない場合、初期値として "1girl" / "1boy" のいずれかを入力\
#@markdown (  "プロンプトアシストを使う" のオプションによって補完されます)

#@markdown  次の機能をオンにします
#@markdown   * 画像の質を上げるプロントを追加する
#@markdown   * プロンプトアシストを使う
#@markdown   * 推奨するネガティブプロントを使用
#@markdown   * 画面に表示



#@markdown

#@markdown -----

# @markdown >生成したい枚数を入力してください　/ Please enter the number of images you want to generate here.
生成する枚数 = 9 #@param {type:"slider", min:1, max:100, step:1}
枚数制限なし = False #@param {type:"boolean"}
#if 生成する枚数 <= 0 or isinstance(生成する枚数, float):
  #  print("\033[31m警告:無効な数字が入力された為デフォルトの1枚に設定しました\033[0m")
  #  生成する枚数 = 1
#if 生成する枚数 is None:
 #   print("\033[31m警告:無効な形式な為デフォルトの1枚に設定しました\033[0m")
#    生成する枚数 = 1

# @markdown  デフォルトでは1回につき1枚生成します。(By default, one image will be generated per run.)

# @markdown ------

# @markdown  >生成する画像の条件を**英語で**入力してください　(Please input the conditions for generating images in **English**.)

プロンプト = "Mass," #@param {type:"string"}
画像の質を上げるプロントを追加する = True #@param {type:"boolean"}
プロンプトアシストを使う = True #@param {type:"boolean"}
日本語入力 = False #@param {type:"boolean"}
# @markdown 例、"Earth, space, high resolution" や "Please draw a beautiful Mount Fuji with the sun rising from the summit."、"cute, cat"、"smail,1girl, white hair, medium hair, cat ears, looking at viewer, :3, cute,white_dress"

# @markdown ------

# @markdown >画像を入力として送リます。

# @markdown 入力形式を "image_and_text" とした場合のみに使えます


入力する画像 = "" #@param {type:"string"}


# @markdown ------

#@markdown #詳細設定

seed値 = None #@param {type:"number"}
guidance_scale = 10 #@param {type:"slider", min:1, max:40, step:1}
温度 = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
#top_k = 17 #@param {type:"slider", min:1, max:40, step:1}
#top_p = 0 #@param {type:"slider", min:0, max:1, step:0.01}
雰囲気 = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}
拡散ステップ = 20  # @param {type:"number"}
#safe_level = "safe" #@param ["safe", "Questionable", "Explicit"]
縦の解像度 = "512" #@param ["480","512","600", "768","800", "1080","1152", "1440", "1920", "3840", "7680"]
横の解像度 = "512" #@param ["480","512","600", "768","800", "1080","1152", "1440", "1920", "3840", "7680"]
height = int(縦の解像度)
width = int(横の解像度)

#@markdown seed(0≦**∞**)=seed値を指定します。何も入力しない・Noneの場合はランダムです。

#@markdown guidance_scale(1≦40 推奨"25")どのくらい細かく描くかを指定します。

#@markdown 温度(0.1≦1.0 推奨0.8)＝どのくらい条件に合わせるかを指定します。

#@markdown 拡散ステップ(1≦1000 推奨400)＝何回計算をするかです。回数を減らすほど生成速度が速くなります

#@markdown 解像度(推奨1080)=解像度を指定します。縦横が異なっていても問題ありません。
#@markdown 数値を小さくしても、あまり生成速度に影響はありません

# @markdown ------

# @markdown >必要であればネガティブプロンプトを入力してください。人物を生成するときにおすすめです

# @markdown よくわからない方ボタンを押してください。おすすめのネガティブプロントを使います

# @markdown (If necessary, please enter a negative prompt. It is recommended for generating characters.)

N_prompt = ""  # @param {type:"string"}
推奨するネガティブプロントを使用 = True #@param {type:"boolean"}
# @markdown ネガティブプロンプトとは、**ネガティブな要素を除く**ものです。 例、low quality、unbeautiful.

# @markdown (Negative prompts are used to exclude negative elements from an image. For example, you can use a negative prompt to exclude low quality images or images that are not beautiful.)

# @markdown ------

#@markdown >保存する先を指定します

# @markdown (If you would like to specify a location to save the generated image, please enter the path.)

保存する先のパス = ""  # @param {type:"string"}
#@markdown デフォルトでは "/content/生成した画像" に保存されます。なければ作るようになっています。

#@markdown (If no path is specified, the images will be saved to "/content/生成した画像". The directory will be created if it doesn't exist.)

# @markdown

# @markdown 補足."/content/生成した画像"は左のタスクバーにあるファイルのマークを押すと"生成した画像"のフォルダを見れます

# @markdown (P.S.You can see the images generated at '/content/生成した画像' by clicking the file icon in the left taskbar.)

# @markdown ------

# @markdown >生成した画像を画面に表示します

画面に表示 = True  # @param {type:"boolean"}
プロンプトを表示 = True  # @param {type:"boolean"}


if 入力形式=="image_and_text":
  try:
    init_image = Image.open(入力する画像)
    init_image = init_image.convert("RGB")
    init_image = init_image.resize((768, 512))
  except:
    raise FileNotFoundError('入力する画像が見つかりませんでした。画像を入力しない場合は、Step.1の入力形式を"text_only"に変更お願いします')

def japanese(Prompt):
  if 日本語入力:
    texts = Prompt
    encoded_text = codecs.encode(texts, 'utf-8')
    decoded_text = codecs.decode(encoded_text, 'utf-8')
    pos_tagger = spacy.load('en_core_web_sm')
    fugu_translator = pipeline('translation', model='staka/fugumt-ja-en')
    translation = fugu_translator(decoded_text, src_lang="ja", tgt_lang="en")[0]["translation_text"]
    prompt_1 = translation #After_Prompt
  else:
    prompt_1 = Prompt
  return prompt_1


def easy_prompt(prompt_1):
  if プロンプトアシストを使う or 自動で条件を決めて生成:
    outs = nlp(prompt_1, max_length=74, num_return_sequences=1, do_sample=True, repetition_penalty=1.2, temperature=0.7, top_k=4, early_stopping=True)
    for i in range(len(outs)):
      # remove trailing commas and double spaces
      outs[i] = str(outs[i]['generated_text']).replace('  ', '').rstrip(',')
      prompt_2 = outs[i]
  else:
    prompt_2 = prompt_1
  return prompt_2

def random_keyword():
    return random.choice(["1girl", "1woman"])

def main_task():
  if プロンプト=="":
    prompt_1=random_keyword()
    prompt_2=easy_prompt(prompt_1)
    print('promptが未入力なため自動で入力します。')
  else:
    prompt_1 = japanese(Prompt)
    prompt_2 = easy_prompt(prompt_1)
  return prompt_2

prompt_2 = main_task()

if not Google_driveに接続:
  if "/content/drive/MyDrive" in 保存する先のパス:
    print("\033[31mGoogleドライブに接続されていないためデフォルトのパスに保存しました\033[0m")

if seed値 is None:
   seed値 = random.randint(1,1000000)


if 画像の質を上げるプロントを追加する or 自動で条件を決めて生成 :
  追加 = "((((masterpiece)))), (((best quality))), high quality, absurdres"
else:
  追加 = ""


if 推奨するネガティブプロントを使用 or 自動で条件を決めて生成 :
  おすすめのネガティブプロント = ",((((worst quality)))), ((((low quality)))), blurred, medium quality, purple hair, deleted, lowres, comic, bad anatomy, (((bad hands))), low resolution, blurry, pixelated, sketchy, nonsensical anatomy, unrealistic pose, mosaic, unclear details, distorted colors, unrealistic proportions, poor quality, fuzzy, out of focus, hazy, grainy, text, error, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, standard quality, bad feet_hand_finger_leg_eye, bad, text font ui, bad shadow, poorly drawn, black-white, ugly, duplicate, mutation, mutilated, malformed mutated, malformed, The background is incoherent, simple background, low-quality background, low background, bad body, long body, broken limb, anatomical nonsense, extra limbs, missing limb, incorrect limb, multiple heads, twisted head, poorly drawn face, head_wings, More than two cows, missing head, 1 unit with multiple heads, heads together, abnormal eye proportion, unclear eyes, bad eyes, fused eyes, poorly drawn eyes, bad mouth, poorly drawn mouth, bad tongue, too long tongue, bad ears, poorly drawn ears, extra ears, heavy ears, long neck, too thick neck, bad neck, abnormally large breasts, extra_breasts, fused breasts, bad breasts, missing arms, disappearing arms, extra arms, three arms, mutated hands and fingers, black hands, fused hand, missing fingers, extra digits, huge thighs, disappearing thigh, missing thighs, extra thighs, bad feet, bad legs, huge calf, disappearing legs, bad gloves, fused gloves, beard, artist name, text watermark, unnatural, obviously wrong, distorted face, floating hair, floating body parts, severed body parts, incorrect leg position, deformed, fused body and hands, disregard of physics, distorted shape, doll-like object not present in the image, body fusion, abnormal fingers, fingers resembling fish fins, dot eyes, unclear background, mosaic, body bending, incorrect leg-to-torso ratio, excessively large breasts, more than three limbs (hands/legs), unsettling appearance, eyes filled with solid color, lack of lower body, splitting, creepy doll-like appearance, distorted eyes, lines on the skin, legs bending in unnatural directions, abnormal finger count, missing arms, floating hands, lack of nose or mouth, hands emerging from impossible places, incorrect body part ratios, bad, longbody, lowres, bad anatomy, bad hands, missing fingers, fat, big boobs, pubic hair, Distorted eye contour, Missing part from the ankles onward, extra digit, fewer digits, split wings, Vampire wings floating in the air, bad wing, cropped"
else:
  おすすめのネガティブプロント =""


NOT_prompt = N_prompt + おすすめのネガティブプロント

negative_prompt2=("((nfsw))"+ NOT_prompt)


new_diffuser_kwargs = {
   "negative_prompt":negative_prompt2,
   "seed": seed値,
   "temperature": 温度,
   "truncation": 雰囲気,
   "diffusion_steps": 拡散ステップ
   }
if 枚数制限なし:
  生成する枚数=10000
  st="∞"
else:
  st=生成する枚数

GE_images_path="/tmp/generated_images"
if not os.path.exists(GE_images_path):
  os.makedirs(GE_images_path)

#returnを使うとfor文が機能しないからdef関数は使えない
def generate_images(st,prompt_2):
    prompt = 追加 + prompt_2

    seed = seed値
    #generator.manual_seed(seed)

    diffuser_kwargs.update(new_diffuser_kwargs)

    i=1
    for i in range(生成する枚数):
       j=i+1
       st=st
       if 入力形式 == "image_and_text":
          with autocast("cuda"):
                result = pipe(prompt, image=init_image, negative_prompt=negative_prompt2, guidance_scale=guidance_scale)
                make_image = result.images[0]  # 生成された画像はリストの最初の要素に格納されている
                if 保存する先のパス:
                  seed=seed値
                  date = datetime.now().strftime("%Y%m%d_%H%M%S")
                  filenames = f"seed({seed})_model({model_name})_ガイダンス({guidance_scale})温度({温度})_雰囲気({雰囲気})_{date}.png" # seed値を含めたファイル名を作成する
                  path = os.path.join(保存する先のパス, filenames)
                  if not os.path.exists(保存する先のパス):
                     os.makedirs(保存する先のパス)
                  make_image.save(path)
                  make_image.save(GE_images_path)
                  print("\033[34m画像生成が完了しました(",j,"/",st,")")
                  print("保存先のパス:(\033[32m "+path+"\033[34m)")
                  print("ファイルの名前:(\033[32m"+filenames+"\033[34m)")
                  print("上記のようになります。ご確認ください\033[0m)")
                  if 画面に表示 or 自動で条件を決めて生成:
                    make_image.show()
                  if プロンプトを表示:
                     print('\033[92mプロンプト:  ' + prompt + '\033[0m\n')
                else:
                  seed=seed値
                  date = datetime.now().strftime("%Y%m%d_%H%M%S")
                  filenames = f"seed({seed})_model({model_name})_ガイダンス({guidance_scale})温度({温度})_雰囲気({雰囲気})_{date}.png" # seed値を含めたファイル名を作成する
                  path = os.path.join('/content/生成した画像', filenames)
                  if not os.path.exists('/content/生成した画像'):
                     os.makedirs('/content/生成した画像')
                  make_image.save(path)
                  make_image.save(GE_images_path)
                  print("\033[34m画像生成が完了しました(",j,"/",st,")")
                  print("保存先のパス:(\033[32m content/生成した画像\033[34m)")
                  print("ファイルの名前:(\033[32m"+ filenames +"\033[34m)")
                  print("上記のようになります。ご確認ください\033[0m)")
                  if 画面に表示 or 自動で条件を決めて生成:
                     make_image.show()
                  if プロンプトを表示:
                     print('\033[92mプロンプト:  ' + prompt + '\033[0m\n')
       else:
          with autocast("cuda"):
          # ユーザーからの最新入力を使用して画像生成(字下げに注意1文字あける)
                result = pipe(prompt, negative_prompt=negative_prompt2, guidance_scale=guidance_scale,)
                make_image = result.images[0]
                if 保存する先のパス:
                  seed=seed値
                  date = datetime.now().strftime("%Y%m%d_%H%M%S")
                  filenames = f"seed({seed})_model({model_name})_ガイダンス({guidance_scale})温度({温度})_雰囲気({雰囲気})_{date}.png" # seed値を含めたファイル名を作成する
                  path = os.path.join(保存する先のパス, filenames)
                  if not os.path.exists(保存する先のパス):
                     os.makedirs(保存する先のパス)
                  make_image.save(path)
                  make_image.save(GE_images_path)
                  print("\033[34m画像生成が完了しました(",j,"/",st,")")
                  print("保存先のパス:(\033[32m "+path+"\033[34m)")
                  print("ファイルの名前:(\033[32m"+filenames+"\033[34m)")
                  print("上記のようになります。ご確認ください\033[0m)")
                  if 画面に表示 or 自動で条件を決めて生成:
                     make_image.show()
                  if プロンプトを表示:
                     print('\033[92mプロンプト:  ' + prompt + '\033[0m\n')
                else:
                  seed=seed値
                  date = datetime.now().strftime("%Y%m%d_%H%M%S")
                  filenames = f"seed({seed})_model({model_name})_ガイダンス({guidance_scale})温度({温度})_雰囲気({雰囲気})_{date}.png" # seed値を含めたファイル名を作成する
                  path = os.path.join('/content/生成した画像', filenames)
                  if not os.path.exists('/content/生成した画像'):
                     os.makedirs('/content/生成した画像')
                  make_image.save(path)
                  make_image.save(GE_images_path)
                  print("\033[34m画像生成が完了しました(",j,"/",st,")")
                  print("保存先のパス:(\033[32m content/生成した画像\033[34m)")
                  print("ファイルの名前:(\033[32m"+ filenames +"\033[34m)")
                  print("上記のようになります。ご確認ください\033[0m)")
                  if 画面に表示 or 自動で条件を決めて生成:
                      make_image.show()
                  if プロンプトを表示:
                     print('\033[92mプロンプト:  ' + prompt + '\033[0m\n')
generate_images(st,prompt_2)

In [ ]:
#@title  {display-mode: "form"}
ランタイムを一時的に切断する = False #@param {type: 'boolean'}
#@markdown 作業を中断するときなどにチェックをつけて実行してください /Please check and execute when interrupting the task.
if ランタイムを一時的に切断する == True:
    from google.colab import runtime
    runtime.unassign()
else:
  print('ランタイムの切断をキャンセルしました')

In [ ]:
#@title  {display-mode: "form"}
#@markdown >zip形式にしてダウンロードします。
from google.colab import files
import shutil
try:
  shutil.make_archive('images', 'zip', '/tmp/generated_images')
  files.download('generated_images.zip')
except:
  print("\033[31m生成した画像が見つかりませんでした\033[0m")

バージョン履歴

2023.5.25 Animeモデルを使用して画像生成をした時に条件の変更が反映されないバグが発生しました

2023.5.26 バグを修正
          画像をプロントとして使えるようになりました。

>説明

フィルターの仕組みは、画像の不適切さを評価するために、0から1の範囲で値を割り当てます。値が0に近いほど不適切な画像の可能性が減ります。また、一般的には0.7以上のスコアと判定されたものは露骨な表現があるとされています。

#**Readme**(日本語版)
#利用規約

本画像生成ノーブックを使用するにあたって、利用規約の内容に全て同意したとみなします。

重要な注意点：
**商用利用はご遠慮ください。**
**画像生成によって起こった問題について、私は一切責任を負いません。**

免責事項:
*  使用にあたっては、自己責任でお願いします。
*  本モデルは予告なく変更・非公開・削除する可能性があります。
*  利用規約は予告なく変更する場合があります。
*  このモデルは、趣味で作成したものであり、商用利用などは意図していません。
*  使用にあたって発生した通信量、電気料金など金銭に関わるものの負担は追い兼ねます
*  Stable Diffusion-Ver2.1やその他の追加ライブラリに関する規約がある場合は、それらも確認することを強くお勧めします。
*  本プロジェクトを利用することにより生じた一切の問題について、私は一切責任を負いません。

ー本プロジェクトとは、本画像生成ノートブックや、githubのページなどをさします
___
#本プロジェクトの説明
Stable Diffusion-Ver2.1をベースにした画像生成ノートブックです。

**使用させていただいたツール**
※2023/5/12時点

-Stable Diffusion-ver2.1

-CLIP

使用させていただいたライブラリ
- Stable Diffusion-ver2.1
- CLIP
- tqdm
- ftfy
- scipy
- regex
- torch
- diffusers
- accelerate
- safetensors
- transformers

---

謝辞

本画像生成ノートブックの作成にあたり、オープンソースのリソースやフリーのツールを使用させていただきました。個人的な利用でしたが、これらのリソースやツールがあったからこそ、本プロジェクトを実現することができました。
この場を借りて、オープンソースのコミュニティや、フリーのツールを提供してくださった方々に感謝の意を表します。素晴らしいツールや技術を提供してくださり、本プロジェクトを支援してくださったことに心から感謝いたします。

#**Readme(English_ver)**
# Terms of Use

By using this image generation notebook, you agree to all the contents of the Terms of Use.

Important Notice:
**Please refrain from using for commercial purposes.**
**I am not responsible for any problems caused by image generation.**

Disclaimer:
* Please use it at your own risk.
* This model may be changed, unpublished, or deleted without notice.
* The terms of use may be changed without notice.
* This model is created for personal use and is not intended for commercial use.
* I will not be responsible for any financial costs incurred, such as communication costs and electricity costs, incurred during use.
* If there are terms and conditions for Stable Diffusion-Ver2.1 and other additional libraries, it is strongly recommended to confirm them as well.
* I am not responsible for any problems caused by using this project.

ー This project refers to the image generation notebook and GitHub pages.

---

# Description of this project

This is an image generation notebook based on Stable Diffusion-Ver2.1.

**Tools used**
* As of 2023/5/12

- Stable Diffusion-ver2.1
- CLIP

**Libraries used**
- Stable Diffusion-ver2.1
- CLIP
- tqdm
- ftfy
- scipy
- regex
- torch
- diffusers
- accelerate
- safetensors
- transformers

---

# Acknowledgements

I used open source resources and free tools to create this image generation notebook. Although it was for personal use, it was only possible to realize this project because of these resources and tools.
I would like to express my gratitude to the open source community and those who provide free tools. I sincerely appreciate your support for this project by providing great tools and technologies.
